# LightGBM with Feature Engineering and Interaction Testing
## Travelers UMC 2025 Competition


In [2]:
# LightGBM with Feature Engineering and Interaction Testing
# Travelers UMC 2025 Competition

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Data handling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import (roc_auc_score, f1_score, precision_score, recall_score, 
                            classification_report, average_precision_score)

# LightGBM
import lightgbm as lgb

# Hyperparameter optimization
import optuna
from optuna.samplers import TPESampler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

# Progress bar
from tqdm import tqdm

print("="*80)
print("LightGBM Model with Feature Engineering and Interaction Testing")
print("="*80)


LightGBM Model with Feature Engineering and Interaction Testing


## Step 1: Load Data


In [3]:
# ============================================================================
# STEP 1: Load Data
# ============================================================================
print("\n" + "="*80)
print("LOADING DATA")
print("="*80)

train_df = pd.read_csv('Data/Training_TriGuard.csv')

print(f"Original data shape: {train_df.shape}")
print(f"\nColumns: {list(train_df.columns)}")
print(f"\nData types:\n{train_df.dtypes}")
print(f"\nMissing values:\n{train_df.isnull().sum().sum()}")
print(f"\nFirst few rows:")
print(train_df.head())



LOADING DATA
Original data shape: (17999, 29)

Columns: ['subrogation', 'claim_number', 'year_of_born', 'gender', 'email_or_tel_available', 'safety_rating', 'annual_income', 'high_education_ind', 'address_change_ind', 'living_status', 'zip_code', 'claim_date', 'claim_day_of_week', 'accident_site', 'past_num_of_claims', 'witness_present_ind', 'liab_prct', 'channel', 'policy_report_filed_ind', 'claim_est_payout', 'vehicle_made_year', 'vehicle_category', 'vehicle_price', 'vehicle_color', 'vehicle_weight', 'age_of_DL', 'accident_type', 'in_network_bodyshop', 'vehicle_mileage']

Data types:
subrogation                  int64
claim_number                 int64
year_of_born                 int64
gender                      object
email_or_tel_available       int64
safety_rating                int64
annual_income                int64
high_education_ind           int64
address_change_ind           int64
living_status               object
zip_code                     int64
claim_date           

## Step 2: Data Preprocessing and Date-based Split


In [4]:
# ============================================================================
# STEP 2: Data Preprocessing and Date-based Split
# ============================================================================
print("\n" + "="*80)
print("DATA PREPROCESSING AND DATE-BASED SPLIT")
print("="*80)

# Convert claim_date to datetime
train_df['claim_date'] = pd.to_datetime(train_df['claim_date'])

# Sort by date
train_df = train_df.sort_values('claim_date').reset_index(drop=True)

# Display date range
print(f"Date range: {train_df['claim_date'].min()} to {train_df['claim_date'].max()}")

# Split by date (e.g., use 80% for training, 20% for validation)
# This prevents data leakage by ensuring future data is not used to predict past
split_date = train_df['claim_date'].quantile(0.8)
print(f"\nSplit date (80th percentile): {split_date}")

train_data = train_df[train_df['claim_date'] < split_date].copy()
val_data = train_df[train_df['claim_date'] >= split_date].copy()

print(f"\nTraining set: {train_data.shape[0]} samples ({train_data['claim_date'].min()} to {train_data['claim_date'].max()})")
print(f"Validation set: {val_data.shape[0]} samples ({val_data['claim_date'].min()} to {val_data['claim_date'].max()})")

# Store claim numbers for later
train_claim_numbers = train_data['claim_number'].copy()
val_claim_numbers = val_data['claim_number'].copy()

# Separate features and target
X_train_raw = train_data.drop(['subrogation', 'claim_number', 'claim_date'], axis=1, errors='ignore')
y_train = train_data['subrogation'].astype(int)

X_val_raw = val_data.drop(['subrogation', 'claim_number', 'claim_date'], axis=1, errors='ignore')
y_val = val_data['subrogation'].astype(int)

print(f"\nTraining features shape: {X_train_raw.shape}")
print(f"Validation features shape: {X_val_raw.shape}")
print(f"\nClass distribution in training:")
print(y_train.value_counts(normalize=True))
print(f"\nClass distribution in validation:")
print(y_val.value_counts(normalize=True))



DATA PREPROCESSING AND DATE-BASED SPLIT
Date range: 2015-01-01 00:00:00 to 2016-12-31 00:00:00

Split date (80th percentile): 2016-08-01 00:00:00

Training set: 14372 samples (2015-01-01 00:00:00 to 2016-07-31 00:00:00)
Validation set: 3627 samples (2016-08-01 00:00:00 to 2016-12-31 00:00:00)

Training features shape: (14372, 26)
Validation features shape: (3627, 26)

Class distribution in training:
subrogation
0    0.772474
1    0.227526
Name: proportion, dtype: float64

Class distribution in validation:
subrogation
0    0.767025
1    0.232975
Name: proportion, dtype: float64


## Step 3: Feature Engineering


In [5]:
# ============================================================================
# STEP 3: Feature Engineering - Handle Categorical and Numerical Features
# ============================================================================
print("\n" + "="*80)
print("FEATURE ENGINEERING")
print("="*80)

def preprocess_features(df):
    """Preprocess features: encode categorical, handle missing values"""
    df_processed = df.copy()
    
    # Identify categorical columns
    categorical_cols = df_processed.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
    
    print(f"Categorical columns: {categorical_cols}")
    print(f"Numerical columns: {len(numerical_cols)}")
    
    # Fill missing values
    for col in numerical_cols:
        if df_processed[col].isnull().sum() > 0:
            df_processed[col].fillna(df_processed[col].median(), inplace=True)
    
    for col in categorical_cols:
        if df_processed[col].isnull().sum() > 0:
            df_processed[col].fillna('Unknown', inplace=True)
    
    # One-hot encode categorical variables
    if categorical_cols:
        df_processed = pd.get_dummies(df_processed, columns=categorical_cols, drop_first=True)
    
    return df_processed, categorical_cols, numerical_cols

# Preprocess training and validation sets
X_train_processed, cat_cols, num_cols = preprocess_features(X_train_raw)
X_val_processed, _, _ = preprocess_features(X_val_raw)

# Ensure both sets have the same columns
common_cols = list(set(X_train_processed.columns) & set(X_val_processed.columns))
X_train_processed = X_train_processed[common_cols]
X_val_processed = X_val_processed[common_cols]

# Add missing columns as zeros
for col in X_train_processed.columns:
    if col not in X_val_processed.columns:
        X_val_processed[col] = 0
for col in X_val_processed.columns:
    if col not in X_train_processed.columns:
        X_train_processed[col] = 0

X_val_processed = X_val_processed[X_train_processed.columns]

print(f"\nAfter preprocessing:")
print(f"Training features shape: {X_train_processed.shape}")
print(f"Validation features shape: {X_val_processed.shape}")
print(f"Feature names: {list(X_train_processed.columns)}")



FEATURE ENGINEERING
Categorical columns: ['gender', 'living_status', 'claim_day_of_week', 'accident_site', 'witness_present_ind', 'channel', 'vehicle_category', 'vehicle_color', 'accident_type', 'in_network_bodyshop']
Numerical columns: 16
Categorical columns: ['gender', 'living_status', 'claim_day_of_week', 'accident_site', 'witness_present_ind', 'channel', 'vehicle_category', 'vehicle_color', 'accident_type', 'in_network_bodyshop']
Numerical columns: 16

After preprocessing:
Training features shape: (14372, 41)
Validation features shape: (3627, 41)
Feature names: ['address_change_ind', 'claim_day_of_week_Sunday', 'vehicle_made_year', 'in_network_bodyshop_yes', 'accident_type_multi_vehicle_unclear', 'witness_present_ind_Y', 'year_of_born', 'age_of_DL', 'channel_Online', 'accident_site_Local', 'liab_prct', 'policy_report_filed_ind', 'living_status_Rent', 'claim_day_of_week_Tuesday', 'channel_Phone', 'vehicle_color_gray', 'past_num_of_claims', 'claim_day_of_week_Saturday', 'vehicle_col

## Step 4: Apply Scaling


In [6]:
# ============================================================================
# STEP 4: Apply Scaling
# ============================================================================
print("\n" + "="*80)
print("APPLYING SCALING")
print("="*80)

# Use RobustScaler (less sensitive to outliers) or StandardScaler
scaler = RobustScaler()

# Fit on training data only
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_processed),
    columns=X_train_processed.columns,
    index=X_train_processed.index
)

# Transform validation data
X_val_scaled = pd.DataFrame(
    scaler.transform(X_val_processed),
    columns=X_val_processed.columns,
    index=X_val_processed.index
)

print(f"Scaled training features shape: {X_train_scaled.shape}")
print(f"Scaled validation features shape: {X_val_scaled.shape}")
print(f"\nScaled feature statistics (first 5 features):")
print(X_train_scaled.iloc[:, :5].describe())



APPLYING SCALING
Scaled training features shape: (14372, 41)
Scaled validation features shape: (3627, 41)

Scaled feature statistics (first 5 features):
       address_change_ind  claim_day_of_week_Sunday  vehicle_made_year  \
count        14372.000000              14372.000000       14372.000000   
mean            -0.407946                  0.140551          -0.138725   
std              0.491470                  0.347570           0.969266   
min             -1.000000                  0.000000          -5.000000   
25%             -1.000000                  0.000000          -0.500000   
50%              0.000000                  0.000000           0.000000   
75%              0.000000                  0.000000           0.500000   
max              0.000000                  1.000000           1.000000   

       in_network_bodyshop_yes  accident_type_multi_vehicle_unclear  
count             14372.000000                         14372.000000  
mean                 -0.274631         

## Step 5: Base LightGBM Model


In [7]:
# ============================================================================
# STEP 5: Base LightGBM Model
# ============================================================================
print("\n" + "="*80)
print("BASE LIGHTGBM MODEL")
print("="*80)

# Base LightGBM parameters with high iterations
base_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'n_estimators': 10000,  # High number of iterations
    'max_depth': -1,
    'min_child_samples': 20,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'is_unbalance': True,
    'random_state': 42
}

print("Training base LightGBM model...")
print(f"Parameters: {base_params}")

# Create LightGBM datasets
train_data_lgb = lgb.Dataset(X_train_scaled, label=y_train)
val_data_lgb = lgb.Dataset(X_val_scaled, label=y_val, reference=train_data_lgb)

# Train with early stopping
base_model = lgb.train(
    base_params,
    train_data_lgb,
    valid_sets=[train_data_lgb, val_data_lgb],
    valid_names=['train', 'val'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=200, verbose=True),
        lgb.log_evaluation(period=100)
    ]
)

# Make predictions (probabilities)
y_train_pred_base_proba = base_model.predict(X_train_scaled, num_iteration=base_model.best_iteration)
y_val_pred_base_proba = base_model.predict(X_val_scaled, num_iteration=base_model.best_iteration)

# Find optimal threshold for F1 score
from sklearn.metrics import f1_score

def find_optimal_threshold(y_true, y_proba):
    """Find threshold that maximizes F1 score"""
    thresholds = np.arange(0.1, 0.9, 0.01)
    f1_scores = []
    for threshold in thresholds:
        y_pred_binary = (y_proba >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred_binary)
        f1_scores.append(f1)
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx]
    optimal_f1 = f1_scores[optimal_idx]
    return optimal_threshold, optimal_f1

# Find threshold to match target number of positives
def find_threshold_for_target_positives(y_proba, target_positives, tolerance=10):
    """Find threshold that results in approximately target_positives number of positive predictions"""
    # Sort probabilities in descending order
    sorted_proba = np.sort(y_proba)[::-1]
    
    # Use the target_positives-th highest probability as threshold
    if target_positives > len(y_proba):
        target_positives = len(y_proba)
    if target_positives < 0:
        target_positives = 0
    
    threshold = sorted_proba[target_positives - 1] if target_positives > 0 else sorted_proba[0] + 0.001
    
    # Apply threshold
    y_pred = (y_proba >= threshold).astype(int)
    actual_positives = y_pred.sum()
    
    # Fine-tune if needed
    if abs(actual_positives - target_positives) > tolerance:
        # Binary search for better threshold
        low_threshold = 0.0
        high_threshold = 1.0
        best_threshold = threshold
        best_diff = abs(actual_positives - target_positives)
        
        for _ in range(50):  # Max 50 iterations
            mid_threshold = (low_threshold + high_threshold) / 2
            y_pred_mid = (y_proba >= mid_threshold).astype(int)
            mid_positives = y_pred_mid.sum()
            diff = abs(mid_positives - target_positives)
            
            if diff < best_diff:
                best_diff = diff
                best_threshold = mid_threshold
            
            if mid_positives < target_positives:
                high_threshold = mid_threshold
            else:
                low_threshold = mid_threshold
            
            if diff <= tolerance:
                break
        
        threshold = best_threshold
        y_pred = (y_proba >= threshold).astype(int)
        actual_positives = y_pred.sum()
    
    return threshold, actual_positives

# Find optimal threshold on validation set
optimal_threshold_base, optimal_f1_base = find_optimal_threshold(y_val, y_val_pred_base_proba)

# Convert to binary predictions
y_train_pred_base = (y_train_pred_base_proba >= optimal_threshold_base).astype(int)
y_val_pred_base = (y_val_pred_base_proba >= optimal_threshold_base).astype(int)

# Calculate metrics
train_f1_base = f1_score(y_train, y_train_pred_base)
val_f1_base = f1_score(y_val, y_val_pred_base)
train_auc_base = roc_auc_score(y_train, y_train_pred_base_proba)
val_auc_base = roc_auc_score(y_val, y_val_pred_base_proba)

print(f"\n" + "="*80)
print("BASE MODEL RESULTS")
print("="*80)
print(f"Training F1: {train_f1_base:.6f}")
print(f"Validation F1: {val_f1_base:.6f}")
print(f"Training AUC: {train_auc_base:.6f}")
print(f"Validation AUC: {val_auc_base:.6f}")
print(f"Optimal threshold: {optimal_threshold_base:.4f}")
print(f"Best iteration: {base_model.best_iteration}")

# Feature importance
feature_importance_base = pd.DataFrame({
    'feature': X_train_scaled.columns,
    'importance': base_model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

print(f"\nTop 10 Most Important Features:")
print(feature_importance_base.head(10))



BASE LIGHTGBM MODEL
Training base LightGBM model...
Parameters: {'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt', 'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'n_estimators': 10000, 'max_depth': -1, 'min_child_samples': 20, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'is_unbalance': True, 'random_state': 42}
Training until validation scores don't improve for 200 rounds
[100]	train's auc: 0.897628	val's auc: 0.837369
[200]	train's auc: 0.936255	val's auc: 0.83723
Early stopping, best iteration is:
[96]	train's auc: 0.895718	val's auc: 0.837971

BASE MODEL RESULTS
Training F1: 0.659226
Validation F1: 0.596457
Training AUC: 0.895718
Validation AUC: 0.837971
Optimal threshold: 0.5500
Best iteration: 96

Top 10 Most Important Features:
                                feature    importance
10                            liab_prct  36295.765280
22             accident_type_single_car  17185.042723
5      

## Step 6: Feature Interaction Testing


In [ ]:
# ============================================================================
# STEP 6: Feature Interaction Testing
# ============================================================================
print("\n" + "="*80)
print("FEATURE INTERACTION TESTING")
print("="*80)

# Get top features for interaction testing (to reduce computation time)
top_n_features = 50  # Test interactions among top 50 features
top_features = feature_importance_base.head(top_n_features)['feature'].tolist()

print(f"Testing interactions among top {top_n_features} features")
print(f"Total interactions to test: {len(top_features) * (len(top_features) - 1) // 2}")

# Function to test feature interactions
def test_feature_interaction(X_train, X_val, y_train, y_val, feature1, feature2, base_f1):
    """Test interaction between two features"""
    X_train_inter = X_train.copy()
    X_val_inter = X_val.copy()
    
    # Create interaction feature (multiplication)
    X_train_inter[f'{feature1}_x_{feature2}'] = X_train[feature1] * X_train[feature2]
    X_val_inter[f'{feature1}_x_{feature2}'] = X_val[feature1] * X_val[feature2]
    
    # Train quick model
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.1,
        'n_estimators': 500,
        'verbose': -1,
        'random_state': 42
    }
    
    train_data_lgb = lgb.Dataset(X_train_inter, label=y_train)
    val_data_lgb = lgb.Dataset(X_val_inter, label=y_val, reference=train_data_lgb)
    
    model = lgb.train(
        params,
        train_data_lgb,
        valid_sets=[val_data_lgb],
        callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
    )
    
    y_val_pred_proba = model.predict(X_val_inter, num_iteration=model.best_iteration)
    
    # Find optimal threshold and calculate F1
    opt_thresh, _ = find_optimal_threshold(y_val, y_val_pred_proba)
    y_val_pred_binary = (y_val_pred_proba >= opt_thresh).astype(int)
    val_f1 = f1_score(y_val, y_val_pred_binary)
    
    improvement = val_f1 - base_f1
    return val_f1, improvement, f'{feature1}_x_{feature2}'

# Test all pairwise interactions with progress bar
print("\nTesting pairwise feature interactions...")
interaction_results = []

# Calculate total number of interactions
total_interactions = len(top_features) * (len(top_features) - 1) // 2

# Use tqdm for progress bar
with tqdm(total=total_interactions, desc="Testing interactions", unit="interaction") as pbar:
    for i, feat1 in enumerate(top_features):
        for feat2 in top_features[i+1:]:
            if feat1 != feat2:
                try:
                    f1, improvement, interaction_name = test_feature_interaction(
                        X_train_scaled, X_val_scaled, y_train, y_val, 
                        feat1, feat2, val_f1_base
                    )
                    interaction_results.append({
                        'feature1': feat1,
                        'feature2': feat2,
                        'interaction': interaction_name,
                        'val_f1': f1,
                        'improvement': improvement
                    })
                except Exception as e:
                    tqdm.write(f"Error testing {feat1} x {feat2}: {e}")
                finally:
                    pbar.update(1)

# Sort by improvement
interaction_df = pd.DataFrame(interaction_results)
interaction_df = interaction_df.sort_values('improvement', ascending=False)

print(f"\nTested {len(interaction_df)} interactions")
print(f"\nTop 50 Best Interactions:")
print(interaction_df.head(50))

# Select interactions that improve performance
min_improvement = 0.001  # Minimum improvement threshold
selected_interactions = interaction_df[interaction_df['improvement'] > min_improvement]

print(f"\nSelected {len(selected_interactions)} interactions with improvement > {min_improvement}")



FEATURE INTERACTION TESTING
Testing interactions among top 50 features
Total interactions to test: 820

Testing pairwise feature interactions...


Testing interactions:   4%|▍         | 33/820 [00:28<12:14,  1.07interaction/s]

## Step 7: Add Selected Interactions to Features


In [ ]:
# ============================================================================
# STEP 7: Add Selected Interactions to Features
# ============================================================================
print("\n" + "="*80)
print("ADDING SELECTED INTERACTIONS")
print("="*80)

# Create new feature sets with interactions
X_train_with_interactions = X_train_scaled.copy()
X_val_with_interactions = X_val_scaled.copy()

added_count = 0
for _, row in selected_interactions.head(50).iterrows():  # Limit to top 50 interactions
    feat1 = row['feature1']
    feat2 = row['feature2']
    interaction_name = row['interaction']
    
    if feat1 in X_train_scaled.columns and feat2 in X_train_scaled.columns:
        X_train_with_interactions[interaction_name] = X_train_scaled[feat1] * X_train_scaled[feat2]
        X_val_with_interactions[interaction_name] = X_val_scaled[feat1] * X_val_scaled[feat2]
        added_count += 1

print(f"Added {added_count} interaction features")
print(f"New feature count: {X_train_with_interactions.shape[1]} (was {X_train_scaled.shape[1]})")


## Step 8: Hyperparameter Tuning with Optuna


In [ ]:
# ============================================================================
# STEP 8: Hyperparameter Tuning with Optuna
# ============================================================================
print("\n" + "="*80)
print("HYPERPARAMETER TUNING WITH OPTUNA")
print("="*80)

def objective(trial):
    """Optuna objective function for hyperparameter tuning - optimizes F1 score"""
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 10, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'verbose': -1,
        'random_state': 42,
        'is_unbalance': True
    }
    
    train_data_lgb = lgb.Dataset(X_train_with_interactions, label=y_train)
    val_data_lgb = lgb.Dataset(X_val_with_interactions, label=y_val, reference=train_data_lgb)
    
    model = lgb.train(
        params,
        train_data_lgb,
        valid_sets=[val_data_lgb],
        num_boost_round=2000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=100, verbose=False),
            lgb.log_evaluation(period=0)
        ]
    )
    
    y_val_pred_proba = model.predict(X_val_with_interactions, num_iteration=model.best_iteration)
    
    # Find optimal threshold and calculate F1 score
    opt_thresh, _ = find_optimal_threshold(y_val, y_val_pred_proba)
    y_val_pred_binary = (y_val_pred_proba >= opt_thresh).astype(int)
    f1 = f1_score(y_val, y_val_pred_binary)
    
    return f1

# Run optimization
print("Starting hyperparameter optimization...")
study = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42)
)

n_trials = 100  # Number of trials for tuning
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

print(f"\nBest hyperparameters:")
print(study.best_params)
print(f"\nBest validation F1: {study.best_value:.6f}")


## Step 9: Train Final Tuned Model


In [ ]:
# ============================================================================
# STEP 9: Train Final Tuned Model
# ============================================================================
print("\n" + "="*80)
print("TRAINING FINAL TUNED MODEL")
print("="*80)

# Get best parameters
best_params = study.best_params.copy()
best_params.update({
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'random_state': 42,
    'is_unbalance': True,
    'n_estimators': 10000  # High number of iterations
})

print(f"Final model parameters: {best_params}")

# Train final model
train_data_lgb = lgb.Dataset(X_train_with_interactions, label=y_train)
val_data_lgb = lgb.Dataset(X_val_with_interactions, label=y_val, reference=train_data_lgb)

final_model = lgb.train(
    best_params,
    train_data_lgb,
    valid_sets=[train_data_lgb, val_data_lgb],
    valid_names=['train', 'val'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=200, verbose=True),
        lgb.log_evaluation(period=100)
    ]
)

# Make predictions (probabilities)
y_train_pred_final_proba = final_model.predict(X_train_with_interactions, num_iteration=final_model.best_iteration)
y_val_pred_final_proba = final_model.predict(X_val_with_interactions, num_iteration=final_model.best_iteration)

# Find optimal threshold for F1 score
optimal_threshold_final, optimal_f1_final = find_optimal_threshold(y_val, y_val_pred_final_proba)

# Convert to binary predictions
y_train_pred_final = (y_train_pred_final_proba >= optimal_threshold_final).astype(int)
y_val_pred_final = (y_val_pred_final_proba >= optimal_threshold_final).astype(int)

# Calculate metrics
train_f1_final = f1_score(y_train, y_train_pred_final)
val_f1_final = f1_score(y_val, y_val_pred_final)
train_auc_final = roc_auc_score(y_train, y_train_pred_final_proba)
val_auc_final = roc_auc_score(y_val, y_val_pred_final_proba)

print(f"\n" + "="*80)
print("FINAL MODEL RESULTS")
print("="*80)
print(f"Training F1: {train_f1_final:.6f}")
print(f"Validation F1: {val_f1_final:.6f}")
print(f"Training AUC: {train_auc_final:.6f}")
print(f"Validation AUC: {val_auc_final:.6f}")
print(f"Optimal threshold: {optimal_threshold_final:.4f}")
print(f"Best iteration: {final_model.best_iteration}")
print(f"\nImprovement over base model (F1): {val_f1_final - val_f1_base:.6f}")

# Feature importance
feature_importance_final = pd.DataFrame({
    'feature': X_train_with_interactions.columns,
    'importance': final_model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

print(f"\nTop 20 Most Important Features (Final Model):")
print(feature_importance_final.head(20))

# Save feature importance
feature_importance_final.to_csv('lightgbm_feature_importance.csv', index=False)
print(f"\nFeature importance saved to: lightgbm_feature_importance.csv")


## Step 10: Model Comparison Summary


In [ ]:
# ============================================================================
# STEP 10: Model Comparison Summary
# ============================================================================
print("\n" + "="*80)
print("MODEL COMPARISON SUMMARY")
print("="*80)

comparison = pd.DataFrame({
    'Model': ['Base LightGBM', 'Tuned LightGBM with Interactions'],
    'Training F1': [train_f1_base, train_f1_final],
    'Validation F1': [val_f1_base, val_f1_final],
    'Training AUC': [train_auc_base, train_auc_final],
    'Validation AUC': [val_auc_base, val_auc_final],
    'F1 Improvement': [0, val_f1_final - val_f1_base]
})

print(comparison.to_string(index=False))

# Visualize feature importance
plt.figure(figsize=(12, 8))
top_features_plot = feature_importance_final.head(50)
plt.barh(range(len(top_features_plot)), top_features_plot['importance'].values)
plt.yticks(range(len(top_features_plot)), top_features_plot['feature'].values)
plt.xlabel('Feature Importance (Gain)')
plt.title('Top 20 Feature Importance - Final LightGBM Model')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('lightgbm_feature_importance.png', dpi=300, bbox_inches='tight')
print(f"\nFeature importance plot saved to: lightgbm_feature_importance.png")
plt.show()


## Step 11: Load Test Data and Generate Predictions


In [ ]:
# ============================================================================
# STEP 11: Load Test Data
# ============================================================================
print("\n" + "="*80)
print("LOADING TEST DATA")
print("="*80)

# Load test data
test_df = pd.read_csv('Data/Testing_TriGuard.csv')
test_original = test_df.copy()

print(f"Test data shape: {test_df.shape}")
print(f"\nTest columns: {list(test_df.columns)}")

# Store claim numbers for submission
test_claim_numbers = test_df['claim_number'].copy()

# Separate features (no target in test data)
X_test_raw = test_df.drop(['claim_number'], axis=1, errors='ignore')

print(f"\nTest features shape: {X_test_raw.shape}")


In [ ]:
# ============================================================================
# STEP 12: Preprocess Test Data (Same as Training)
# ============================================================================
print("\n" + "="*80)
print("PREPROCESSING TEST DATA")
print("="*80)

# Apply the same preprocessing function
X_test_processed, _, _ = preprocess_features(X_test_raw)

# Ensure test set has the same columns as training
# Add missing columns as zeros
for col in X_train_processed.columns:
    if col not in X_test_processed.columns:
        X_test_processed[col] = 0

# Remove columns that don't exist in training
X_test_processed = X_test_processed[X_train_processed.columns]

print(f"Test features after preprocessing: {X_test_processed.shape}")
print(f"Training features shape: {X_train_processed.shape}")


In [ ]:
# ============================================================================
# STEP 13: Scale Test Data
# ============================================================================
print("\n" + "="*80)
print("SCALING TEST DATA")
print("="*80)

# Use the same scaler fitted on training data
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_processed),
    columns=X_test_processed.columns,
    index=X_test_processed.index
)

print(f"Scaled test features shape: {X_test_scaled.shape}")


In [ ]:
# ============================================================================
# STEP 14: Add Interaction Features to Test Data
# ============================================================================
print("\n" + "="*80)
print("ADDING INTERACTION FEATURES TO TEST DATA")
print("="*80)

# Create test feature set with interactions (same as training)
X_test_with_interactions = X_test_scaled.copy()

added_count = 0
for _, row in selected_interactions.head(30).iterrows():  # Same interactions as training
    feat1 = row['feature1']
    feat2 = row['feature2']
    interaction_name = row['interaction']
    
    if feat1 in X_test_scaled.columns and feat2 in X_test_scaled.columns:
        X_test_with_interactions[interaction_name] = X_test_scaled[feat1] * X_test_scaled[feat2]
        added_count += 1

print(f"Added {added_count} interaction features to test data")
print(f"Test features with interactions: {X_test_with_interactions.shape[1]} features")

# Ensure all columns match training data
for col in X_train_with_interactions.columns:
    if col not in X_test_with_interactions.columns:
        X_test_with_interactions[col] = 0

# Select only columns that exist in training
X_test_with_interactions = X_test_with_interactions[X_train_with_interactions.columns]

print(f"Final test features shape: {X_test_with_interactions.shape}")
print(f"Training features shape: {X_train_with_interactions.shape}")


In [ ]:
# ============================================================================
# STEP 15: Select Top 50 Features
# ============================================================================
print("\n" + "="*80)
print("SELECTING TOP 50 FEATURES")
print("="*80)

# Get top 50 features based on feature importance from final model
top_50_features = feature_importance_final.head(50)['feature'].tolist()

print(f"Top 50 features selected:")
print(top_50_features)

# Select these features for both training and test
X_train_top50 = X_train_with_interactions[top_50_features]
X_test_top50 = X_test_with_interactions[top_50_features]

print(f"\nTraining features (top 50): {X_train_top50.shape}")
print(f"Test features (top 50): {X_test_top50.shape}")


In [ ]:
# ============================================================================
# STEP 16: Retrain Model on Top 50 Features
# ============================================================================
print("\n" + "="*80)
print("RETRAINING MODEL ON TOP 50 FEATURES")
print("="*80)

# Use the best parameters from tuning
best_params_top50 = best_params.copy()

print(f"Training with parameters: {best_params_top50}")

# Create LightGBM datasets with top 50 features
train_data_lgb_top50 = lgb.Dataset(X_train_top50, label=y_train)
val_data_lgb_top50 = lgb.Dataset(X_val_with_interactions[top_50_features], label=y_val, reference=train_data_lgb_top50)

# Retrain model on top 50 features
final_model_top50 = lgb.train(
    best_params_top50,
    train_data_lgb_top50,
    valid_sets=[train_data_lgb_top50, val_data_lgb_top50],
    valid_names=['train', 'val'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=200, verbose=True),
        lgb.log_evaluation(period=100)
    ]
)

# Evaluate on validation set
y_val_pred_top50_proba = final_model_top50.predict(X_val_with_interactions[top_50_features], num_iteration=final_model_top50.best_iteration)

# Find optimal threshold for F1 score
optimal_threshold_top50, optimal_f1_top50 = find_optimal_threshold(y_val, y_val_pred_top50_proba)

# Convert to binary predictions
y_val_pred_top50 = (y_val_pred_top50_proba >= optimal_threshold_top50).astype(int)
val_f1_top50 = f1_score(y_val, y_val_pred_top50)
val_auc_top50 = roc_auc_score(y_val, y_val_pred_top50_proba)

print(f"\n" + "="*80)
print("TOP 50 FEATURE MODEL RESULTS")
print("="*80)
print(f"Validation F1 (top 50 features): {val_f1_top50:.6f}")
print(f"Validation AUC (top 50 features): {val_auc_top50:.6f}")
print(f"Optimal threshold: {optimal_threshold_top50:.4f}")
print(f"Best iteration: {final_model_top50.best_iteration}")


In [ ]:
# ============================================================================
# STEP 17: Generate Predictions on Test Data
# ============================================================================
print("\n" + "="*80)
print("GENERATING PREDICTIONS ON TEST DATA")
print("="*80)

# Make predictions on test data (probabilities)
test_predictions_proba = final_model_top50.predict(X_test_top50, num_iteration=final_model_top50.best_iteration)

# Target number of positives from Kaggle probing: ~2778 (23.15% of 12000)
target_positives = 2778

# Find threshold that results in approximately target_positives
threshold_for_target, actual_positives = find_threshold_for_target_positives(
    test_predictions_proba, 
    target_positives, 
    tolerance=5
)

# Also check F1-optimized threshold for comparison
test_predictions_f1 = (test_predictions_proba >= optimal_threshold_top50).astype(int)
positives_f1 = test_predictions_f1.sum()

print(f"Test predictions shape: {test_predictions_proba.shape}")
print(f"\nThreshold Selection:")
print(f"  F1-optimized threshold: {optimal_threshold_top50:.6f} -> {positives_f1} positives ({positives_f1/len(test_predictions_proba)*100:.2f}%)")
print(f"  Target-matching threshold: {threshold_for_target:.6f} -> {actual_positives} positives ({actual_positives/len(test_predictions_proba)*100:.2f}%)")
print(f"  Target: {target_positives} positives (23.15%)")
print(f"  Difference from target: {actual_positives - target_positives} instances")

# Use target-matching threshold for final predictions
test_predictions = (test_predictions_proba >= threshold_for_target).astype(int)

print(f"\nFinal binary predictions:")
print(f"  Range: [{test_predictions.min()}, {test_predictions.max()}]")
print(f"  Predicted positives: {test_predictions.sum()} ({test_predictions.sum()/len(test_predictions)*100:.2f}%)")
print(f"  Predicted negatives: {(test_predictions == 0).sum()} ({(test_predictions == 0).sum()/len(test_predictions)*100:.2f}%)")
print(f"\nFirst 10 binary predictions:")
print(test_predictions[:10])
print(f"\nFirst 10 probabilities:")
print(test_predictions_proba[:10])


In [ ]:
# ============================================================================
# STEP 18: Generate Submission CSV
# ============================================================================
print("\n" + "="*80)
print("GENERATING SUBMISSION CSV")
print("="*80)

# Create submission DataFrame with binary predictions (0 or 1)
submission = pd.DataFrame({
    'claim_number': test_claim_numbers,
    'subrogation': test_predictions.astype(int)  # Ensure integer type
})

# Save submission file
submission_filename = 'lightgbm_submission.csv'
submission.to_csv(submission_filename, index=False)

print(f"Submission file saved: {submission_filename}")
print(f"\nSubmission statistics:")
print(f"  Shape: {submission.shape}")
print(f"  Prediction range: [{submission['subrogation'].min()}, {submission['subrogation'].max()}]")
print(f"  Predicted positives: {submission['subrogation'].sum()} ({submission['subrogation'].sum()/len(submission)*100:.2f}%)")
print(f"  Predicted negatives: {(submission['subrogation'] == 0).sum()} ({(submission['subrogation'] == 0).sum()/len(submission)*100:.2f}%)")
print(f"  Expected positives (from Kaggle): ~2,778 (23.15%)")
print(f"  Difference: {submission['subrogation'].sum() - 2778} instances")
print(f"\nFirst 10 rows:")
print(submission.head(10))
print(f"\nLast 10 rows:")
print(submission.tail(10))

# Verify submission format matches sample
sample_submission = pd.read_csv('Data/sample_submission.csv')
print(f"\nSample submission format:")
print(sample_submission.head())
print(f"\nSubmission format matches: {list(submission.columns) == list(sample_submission.columns)}")
print(f"Values are binary (0 or 1): {submission['subrogation'].isin([0, 1]).all()}")
